In [30]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [31]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [28]:
# s3 = boto3.resource('s3')
# bucket = s3.Bucket('demo-s3-nhhung2')
# prefix = 'data/2022-01-03'

# list_files_in_prefix(bucket, prefix)

['data/2022-01-03/2022-01-03_BINS_XETR08.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR09.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR10.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR11.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR12.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR13.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR14.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR15.csv',
 'data/2022-01-03/2022-01-03_BINS_XETR16.csv']

In [32]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True
    

In [33]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
    return return_date_list

In [29]:
# Application Layer - not core

# bucket= s3.Bucket('demo-s3-nhhung2')
# arg_date = '2022-01-04'
# src_format = '%Y-%m-%d'

# def return_date_list(bucket, arg_date, src_format):
#     min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
#     today = datetime.today().date()
#     return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
#     return return_date_list

# return_date_list(bucket, arg_date, src_format)
# a= return_date_list(bucket, arg_date, src_format)
# b=[]
# for i in a:
#     k='data/'+ i;
#     b.append(k)

# date_list=b

# def extract(bucket, date_list):
#     files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
#     df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
#     return df   

# s3 = boto3.resource('s3')
# bucket= s3.Bucket('demo-s3-nhhung2')
# date_list=b

# extract(bucket, date_list)
# def extract(bucket, date_list):
#     files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
#     df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
#     return df    
    
    

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035814,DE000A1J5RX9,O2D,TELEFONICA DTLD HLDG NA,Common stock,EUR,2504492,2022-01-25,16:44,2.437,2.437,2.437,2.437,20000,2
2035815,DE0006204407,KU2,KUKA AG,Common stock,EUR,2505017,2022-01-25,16:44,72.800,72.800,72.800,72.800,159,3
2035816,DE0008232125,LHA,LUFTHANSA AG VNA O.N.,Common stock,EUR,2505130,2022-01-25,16:44,6.677,6.677,6.677,6.677,440,2
2035817,US88160R1014,TL0,"TESLA INC. DL -,001",Common stock,EUR,2506575,2022-01-25,16:44,809.000,809.000,809.000,809.000,8,2


In [41]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2022-01-04'
    src_format = '%Y-%m-%d'
    src_bucket = 'demo-s3-nhhung2'
    trg_bucket = 'xetra-112233'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    
    date_lst = return_date_list(bucket_src, arg_date, src_format)
    b=[]
    for i in date_lst:
        k='data/'+ i;
        b.append(k)

    date_list=b  
    # date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

In [40]:
# run

main()

## Reading the uploaded file

In [42]:
trg_bucket = 'xetra-112233'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20231216_105737.parquet
xetra_daily_report_20231216_171323.parquet
xetra_daily_report_20231218_160058.parquet
xetra_daily_report_20240122_202734.parquet
xetra_daily_report_20240122_204801.parquet
xetra_daily_report_20240124_095750.parquet
xetra_daily_report_20240124_123654.parquet
xetra_daily_report_20240124_124746.parquet


In [45]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20240124_124746.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [46]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
54309,XS2376095068,2022-01-19,36.29,36.76,36.29,37.09,0,0.80
54310,XS2376095068,2022-01-20,36.89,38.13,36.89,38.13,54,3.74
54311,XS2376095068,2022-01-21,34.55,34.14,33.98,34.55,1857,-10.47
54312,XS2376095068,2022-01-24,30.94,30.26,29.39,31.07,5750,-11.35
